In [178]:
import pandas as pd
import PyPDF2
import openai
from openai import OpenAI 
import os
import csv 
import json
import re
import time

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


# openai_api_key  = os.environ['OPENAI_API_KEY']


openai_api_key = os.getenv('your_api_key')
if not openai_api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in environment.")

In [179]:
#Syllabi file location 
pdf_file_path= 'your_file_path'
#EXtract text from PDF
def extract_text(pdf_file_path):
    text= ''
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()    

    return text

In [180]:
# Store extradcted text
extracted_text = extract_text(pdf_file_path)

In [181]:
# Answer File Location
answer_file_path = 'your_file_path'
answer_df=pd.read_csv(answer_file_path)

In [182]:
answer_df.head()

,Category,Subcategory,Specifics,SpecificsExplained,Id,Size
0,Engagement Opportunities,Course Materials,List reading materials,List required reading materials for the coruse,List-reading-materials,1
1,Engagement Opportunities,Pedagogy,In-class discussion,Have in-class discussion activities,In-class-discussion,1
2,Engagement Opportunities,Pedagogy,Out-of-class discussion,Have out-of-class discussion activities such a...,Out-of-class-discussion,1
3,Engagement Opportunities,Use of Technology,Frequent use of LMS,Uses LMS frequently for the course,Frequent-use-of-LMS,1
4,Engagement Opportunities,Pedagogy,Connect to authentic examples,Encoruage students to connect course content w...,Connect-to-authentic-examples,1


In [183]:
questions_eo={
    "q_eo1": "Is there a list of reading materials that will be used/discussed in class?",
    "q_eo2": "Does it have any in-class discussions or activities that involve discussion?",
    "q_eo3": "Does it have any out-of-class discussion activities? (e.g. Discussion board, Packback)",
    "q_eo4": "Is a Learning Management System (e.g. Canvas or Blackboard) used frequently in the course?",
    "q_eo5": "Does the course explore real-world problems related to class content?",
    "q_eo6": "Does it describe assingments or class activities that encourage students to ask questions and use course content to create projects?",
    "q_eo7": "Are there any in-class assessment of learning opportunities, such as using Tophat or Clicker for polling?", 
    "q_eo8": "Is there any extra credit opportunities for this class?",
    "q_eo9": "Are quizzes and exams adopt short answer or essey questions mainly to engage students?", 
    "q_eo10": "Does the course objectives/description connect to real-life situations?",
    "q_eo11": "Are the course objectives/goals concrete and achievable to students?",
    "q_eo12": "Can the course objective/goals be useful outside of the course?"
}

In [184]:
#revised prompt
client = openai.OpenAI()

def ask_questions(questions, text):
    for question in questions:
        prompt = f'''
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        Do not assume any context from previous documents. This is a fresh evaluation. All evidence must be contained in the provided text.
        '''
        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"Document Content: {text}\n Questions: {questions}"},
        ]
        # get answers from gpt
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # or use gpt-4-1106-preview for better result
            messages=messages,
            temperature=0,
        )
        time.sleep(1.5)  # Adjust this value if needed
        answer = response.choices[0].message.content
    return answer
        

In [185]:
answers_eo_json=ask_questions(questions_eo, extracted_text)
print(answers_eo_json)

{
    "q_eo1": {"answer": "No", "reference": "Not found in the document."},
    "q_eo2": {"answer": "Yes", "reference": "Many classes will be discussion-based."},
    "q_eo3": {"answer": "Yes", "reference": "Weekly assignments will provide practice on what has been discussed in the course."},
    "q_eo4": {"answer": "Yes", "reference": "There will be a schedule of these meetings on Canvas."},
    "q_eo5": {"answer": "Yes", "reference": "The project will involve investigating and proposing enhancements to the department’s curriculum."},
    "q_eo6": {"answer": "Yes", "reference": "Assignments from the first week of the semester will guide students in the process of finding a problem and addressing it."},
    "q_eo7": {"answer": "No", "reference": "Not found in the document."},
    "q_eo8": {"answer": "No", "reference": "Not found in the document."},
    "q_eo9": {"answer": "No", "reference": "Not found in the document."},
    "q_eo10": {"answer": "Yes", "reference": "Students will explo

In [186]:
answers_eo_dic=json.loads(answers_eo_json)
answers_eo_YN={key: value["answer"] for key, value in answers_eo_dic.items()}
answers_eo_YN=json.dumps(answers_eo_YN, indent=2)
print(answers_eo_YN)

{
  "q_eo1": "No",
  "q_eo2": "Yes",
  "q_eo3": "Yes",
  "q_eo4": "Yes",
  "q_eo5": "Yes",
  "q_eo6": "Yes",
  "q_eo7": "No",
  "q_eo8": "No",
  "q_eo9": "No",
  "q_eo10": "Yes",
  "q_eo11": "Yes",
  "q_eo12": "Yes"
}


In [187]:
answers_eo_boolean=[]
answer_eo=json.loads(answers_eo_YN)
for value in answer_eo.values():
    if value.lower() == "yes":
        answers_eo_boolean.append(1)
    elif value.lower() == "no":
        answers_eo_boolean.append(0)
print(answers_eo_boolean)

[0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]


In [188]:
questions_ex={
    "q_ex1": "Does it contain learning material in addition to the required text?",
    "q_ex2": "Are there any opportunities allow students to explore topics that interest them?", 
    "q_ex3": "Does the course explore real-world problems related to class content?",
    "q_ex4": "Is there any evidence of interdiciplinary exploration? such as opportunities in connecting ideas, disciplines, perspectives, contexts. \
            Or does it use the following verbs in the description Associate, Concept map, Connect, Differentiate, Relate, Combine, Contrast/ compare, Correlate, Link, Synthesize",
    "q_ex5": "Are there any learning opportunities that allow students to engage with course material and content at a deeper level (e.g. reflections, writing, projects)?",
    "q_ex6": "Are there any examples used in the course that provide real life examples?"
}

In [189]:
def ask_questions(questions, text):


    for question in questions:
        prompt = f'''
        Instructions:
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        Do not assume any context from previous documents. This is a fresh evaluation. All evidence must be contained in the provided text.
        '''
        
        messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Document Content: {extracted_text}\n Question about the document: {questions}"},
            ]
        #get answers from gpt
        response=client.chat.completions.create(
            model="gpt-4o-mini", # or use gpt-3.5 for lower cost
            messages=messages,
            seed=123,
            temperature=0,
        )
        
        time.sleep(1.5)  # Adjust this value if needed
        answer=response.choices[0].message.content

    return answer    
        

In [190]:
answers_ex_json=ask_questions(questions_ex, extracted_text)
print(answers_ex_json)

{
    "q_ex1": {"answer": "No", "reference": "Not found in the document."},
    "q_ex2": {"answer": "Yes", "reference": "Students will explore modern research topics and career opportunities."},
    "q_ex3": {"answer": "Yes", "reference": "The course project will involve investigating and proposing enhancements to the physics curriculum, informed by interviews with industry professionals."},
    "q_ex4": {"answer": "No", "reference": "Not found in the document."},
    "q_ex5": {"answer": "Yes", "reference": "Weekly assignments will develop broadly applicable skills and involve a major collaborative project."},
    "q_ex6": {"answer": "Yes", "reference": "The course project will involve investigating and proposing enhancements to the department’s curriculum, informed by interviews with industry professionals."}
}


In [191]:
answers_ex_dic=json.loads(answers_ex_json)
answers_ex_YN={key: value["answer"] for key, value in answers_ex_dic.items()}
answers_ex_YN=json.dumps(answers_ex_YN, indent=2)
print(answers_ex_YN)

{
  "q_ex1": "No",
  "q_ex2": "Yes",
  "q_ex3": "Yes",
  "q_ex4": "No",
  "q_ex5": "Yes",
  "q_ex6": "Yes"
}


In [192]:
answers_ex_boolean=[]
answer_ex=json.loads(answers_ex_YN)
for value in answer_ex.values():
    if value.lower() == "yes":
        answers_ex_boolean.append(1)
    elif value.lower() == "no":
        answers_ex_boolean.append(0)
print(answers_ex_boolean)

[0, 1, 1, 0, 1, 1]


In [193]:
questions_gm={ 
    "q_gm1":"Does the syllabus contain language normalizes challenges in this course? It contains language that the instructor beleives students can be successful regardless of the challenges,\
            or share personal or other students' stories of being challenged but succeeded in a course or program in the past. Language related to recognizing student having life situations happening to them does not meet this requirement.",
    "q_gm2":"Is there any opportunity for self-check or practice quizzes and/or exams?",
    "q_gm3":"Do quizzes and/or exams have multiple attempts?",
    "q_gm4":"For writing assingments (e.g. essays, reports, reflections, papers) given in the course, do students have opportunities to get feedback for improvement before receiving the final grades?",
    "q_gm5":"For individual/group projects given in the course, do students have opportunities to get feedback for improvements before receiving the final grades?",
    "q_gm6":"Is there frequent knowledge checks (e.g. homeworks, in-class writing, quizzes, etc.) to assess learning beyond midterm and final exam and final projects?",
    "q_gm7":"Is there any introspective learning opportunities, such as short writing assingments or reflection beyond exams?",
    "q_gm8":"Is there any collaborative learning opportunities(e.g. group projects, peer-grading) for students to learn from their peers?",
    "q_gm9":"Does this syllabus include course objectives that use action verbs (i.e., sketch, solve, train, use, etc.)?",
    "q_gm10":"Do the course objectives seem to promote a focus on learning (e.g., use student centered language, use supportive and collaborative language, encourage students to see challenge as opportunity, use words like 'learning' and 'knowledge', etc.)?",
    "q_gm11":"Outside of course objectives, overall, does it appear that students are encouraged to engage with the course material for the sake of learning?"
}

In [194]:
def ask_questions(questions, text):


    for question in questions:
        prompt=f'''
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        Do not assume any context from previous documents. This is a fresh evaluation. All evidence must be contained in the provided text.        
        '''
        messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Document Content: {extracted_text}\n Question about the document: {questions}"},
            ]
        #get answers from gpt
        response=client.chat.completions.create(
            model="gpt-4o-mini", # or use gpt-4-1106-preview for better result
            messages=messages,
            seed=123,
            temperature=0,
        )
        
        time.sleep(1.5)  # Adjust this value if needed
        answer=response.choices[0].message.content

    return answer    
        

In [195]:
answers_gm_json=ask_questions(questions_gm, extracted_text)
print(answers_gm_json)

{
    "q_gm1": {
        "answer": "Yes",
        "reference": "The document states, 'I strongly believe that all students should be able to participate in the learning process regardless of their ability to purchase course materials' and 'if there is anything that is preventing you from succeeding or anything that you think I could do to help you, please ask.'"
    },
    "q_gm2": {
        "answer": "No",
        "reference": "Not found in the document."
    },
    "q_gm3": {
        "answer": "No",
        "reference": "Not found in the document."
    },
    "q_gm4": {
        "answer": "Yes",
        "reference": "The document mentions that 'grades will be rubric based' and emphasizes the importance of feedback on assignments."
    },
    "q_gm5": {
        "answer": "Yes",
        "reference": "The document states that peer evaluation is part of the course, which allows for feedback on group projects before final grades."
    },
    "q_gm6": {
        "answer": "Yes",
        "ref

In [196]:
answers_gm_dic=json.loads(answers_gm_json)
answers_gm_YN={key: value["answer"] for key, value in answers_gm_dic.items()}
answers_gm_YN=json.dumps(answers_gm_YN, indent=2)
print(answers_gm_YN)

{
  "q_gm1": "Yes",
  "q_gm2": "No",
  "q_gm3": "No",
  "q_gm4": "Yes",
  "q_gm5": "Yes",
  "q_gm6": "Yes",
  "q_gm7": "No",
  "q_gm8": "Yes",
  "q_gm9": "No",
  "q_gm10": "Yes",
  "q_gm11": "Yes"
}


In [197]:
answers_gm_boolean=[]
answer_gm=json.loads(answers_gm_YN)
for value in answer_gm.values():
    if value.lower() == "yes":
        answers_gm_boolean.append(1)
    elif value.lower() == "no":
        answers_gm_boolean.append(0)
print(answers_gm_boolean)

[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1]


In [198]:
# questions_la_pre={
#                   "Step 1: Identify the courese's grading scale and print the items and their associated points first.",
#                   "Step 2: Add a Weight colum to the table, and calcuate each category and their percentage of it in the total course points(weight)",
#                   "Step 3: Reorder the course's grading scale and associated points and weight in decending order, meaning from thie highest points to the lowest points, print it out again.",
#                   "Step 4: With the reordered grading scale identified in Step 3, adding the points up, from the second highest valued category through the last category, save it as SubSum for answering future questions, print it.",
#                   "Step 5: Identify the category has the highest points, and save the points associated with it as Most, print Most=(its numeric value).",
#                   "Step 6: Identify the Total points of the course, and devide Total by 2, save it as Half, print Half=Total/2.",
#                   "Step 7: Identify group-related activities (e.g. discussions in-class or out-of-class, group projects, group presentations, and other opportunities to collaborate and learn from peers)\
#                               and add up the associated points and save them as Group, print it.",
#                   "Step 8: Identify individual related activities (e.g., quizzes, learning checks, homework, TopHat, clickers), and add up the points of all individual activities and save it as Individual.",
#                   "Step 8: Identify the total points for exams, and save it as Exam, print it. If there is no Exam in the course, consider its total points as 0, print it.",
#                   "Step 10: Identify the low stakes assessments, meaning the assessment category that weights 20% or less of the total points, and print them."
#                  }


In [199]:
def get_math(text):
    prompt = f'''
       Instructions:
    You have been provided with the course grading scale data. Your task is to analyze the data, perform the required calculations, and output the results in JSON format.  
    🚫 **Memory should be cleared each time you run this analysis. Do not use previous runs as reference. Fully recalculate each time.**  
    📌 Important: You are looking for a breakdown of **how different assignments or activities contribute to the course grade** (e.g., exams, quizzes, papers). 
    🛑 Do not stop the analysis after a first pass. If you initially believe there is no assignment breakdown, recheck to ensure you didn’t overlook narrative or informal formats.
       Only after completing a structured scan, if you find absolutely no assignment-based grading breakdown, write:
    "No assignment-based grading breakdown was found after full document scan."

    
    --- STEP-BY-STEP PROCESS ---
    1️⃣ **Identify the grading scale items** and their associated points. Print out a table with all items and points.  
    2️⃣ **Add a 'Weight' column**: Calculate each category's percentage of the total course points. Print the updated table with weights.  
    3️⃣ **Reorder the categories** in descending order of points. Print the sorted list.  
    4️⃣ **Calculate 'SubSum'**: Sum all categories from the second-highest to the lowest. Exclude the highest.  
        - Print `SubSum = [value]`.  
    5️⃣ **Identify the highest category** and save it as `'Most'`.  
        - Print `Most = [value]`.  
    6️⃣ **Calculate 'Half'**: Divide the total course points by 2.  
        - Print `Half = [value]`.  
    7️⃣ **Calculate 'Group'**:  
        - Group activities include: "discussions", "group projects", "group presentations", and collaborative activities.  
        - If the title contains "Group", "Team", or "Collaborative", it is included.  
        - Sum these points as `'Group'`.  
        - Print `Group = [value]`.  
    8️⃣ **Calculate 'Individual'**:  
        - Individual activities include: "quizzes", "homework", "learning checks", "papers", "writing assignments", "TopHat", "clickers".  
        - **Exclude any activity labeled as an "Exam"**.  
        - Sum these points as `'Individual'`.  
        - Print `Individual = [value]`.  
    9️⃣ **Calculate 'Exam'**:  
        - Exams include any activity labeled with "Exam", "Test", "Midterm", "Final", or similar terms.  
        - **If it is listed as "Final Exam" or "Midterm", it belongs here, not Individual.**  
        - Sum these points and save as `'Exam'`.  
        - Print `Exam = [value]`.  
    
    --- VERIFICATION STEP ---
    🔍 **Before final output, double-check each calculation:**  
        - Reprint each value and its source data for reference.  
        - If discrepancies are found, **correct them immediately**.  
        - If there is any ambiguity in categorization, explain it and recalculate.  
    
    --- OUTPUT FORMAT ---
    📝 Format the output in JSON as follows:  
        {{
          "SubSum": [SubSum_value],
          "Most": [Most_value],
          "Half": [Half_value],
          "Group": [Group_value],
          "Individual": [Individual_value],
          "Exam": [Exam_value],
          "LowStakes": [List of categories with <= 20% weight]
        }}  
    
    **Do not add backticks, markdown, or "json" in the output. Only raw JSON is allowed.**  
    
    --- MEMORY RESET ---  
🚫 **Ensure that all steps are recalculated fresh every time. Do not reference prior calculations from previous attempts. If there was an error, it must be fixed in the next run.**

    '''

    # Send the message to the LLM model to process the question and return the response
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": text},
    ]

    # Get answers from GPT
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or use gpt-4-1106-preview for better results
        messages=messages,
        temperature=0,
    )
    
    time.sleep(1.5)  # Adjust this value if needed
    answer = response.choices[0].message.content
    return answer


In [200]:
answers_lapre=get_math(extracted_text)
print(answers_lapre)

No assignment-based grading breakdown was found after full document scan.


In [201]:
# 1.If there is a breakdown of assessment categories? Answer Yes or No first. Identify the breakdown of assessment categories and the percentage it weights for the total value of the course, order it by descending order, and use it to answer the following 3 questions. Do group-related activities (e.g., discussions--in or out of class, collaborating or learning from peers) have as much weight (or more) than the exams/mid-terms and final exam?
# 2.Do individual activities (e.g., quizzes, learning checks, homework, TopHat, clickers)have as much weight (or more) than the exams/mid-terms and final exam?
# 3.What category of assessment worth the most based on the grading scale? Is this category of assessment an individual or group-based assessment? Group based return answer Yes, individual return answer No."
# 4.Are the exams cumulative, meaning testing on all knowledge and material students learned in the class all the time?
# 5.Are assessment from the second category to the last category added up together worth more than the highest valued category?
# 6.Is there a method used to assess students’ current knowledge of course concepts (e.g., pre-test, knowledge check, quiz, etc.)?
# 7.Does the syllabi provide flexibility in grading, such as multiple attempts, flexibility in deadlines, dropping an assessment?

questions_la_1={
    "q_la1": "Step 1: State the values of 'SubSum' and 'Most'. Step 2: Compare them explicitly: Is 275 > 225? → Yes. Step 3: Based on the comparison, link the final answer directly to the result. If the comparison is → Yes, then respond 'Yes'. If the comparison is → No, then respond 'No'.",
    "q_la2": "Compare 'Group' to 'Exam'. If 'Group' is more in value, respond with 'Yes'. If 'Group' is less in value, respond with 'No'.", 
    "q_la3": "Evaluate if 'Individual' is more than 'Exam' in value. If it is, respond with 'Yes'. If 'Individual' is less than 'Exam', respond with 'No'.",
    "q_la4": "Step 1: State the value of 'Exam'. Step 2: State the value of 'Half'. Step 3: Perform the comparison: 'Is Exam > Half? → [Yes/No]'. Step 4: Based on the comparison, link the final answer directly to the result. If the comparison is → Yes, then respond 'Yes'. If the comparison is → No, then respond 'No'.", 
    "q_la5": "Determine if low stakes assessments are administered throughout the semester. If they are, respond with 'Yes'. If they are not given throughout the semester, respond with 'No'."
}

In [202]:
def ask_questions(questions, text, answers_lapre):
    # Convert the extracted values from Step 1 to a string (or JSON format)
    step1_data_str = str(answers_lapre)  # This includes all extracted values from Step 1, such as SubSum, Most, etc.

    prompt = f'''
        Instructions:
        You are a teaching assistant. Your task is to answer questions about the grades based on the reference data created in the previous step.
        
        --- PROVIDED VALUES --- 
        {step1_data_str}
        
        --- IMPORTANT RULES ---
        1️⃣ Only use the values given in the reference data.  
        2️⃣ Do **not** remember previous evaluations. This is a fresh analysis every time.  
        3️⃣ Perform all calculations **step-by-step** and print them out.  
        4️⃣ If a comparison is made, print the exact calculation:
            Example: "Is 50 > 30? → Yes"  
            Example: "Is 25 < 30? → No"  
        
        --- LOGIC AND VERIFICATION ---
        - For each question:
            1. Identify the values from the reference data.
            2. Perform the comparison explicitly and display it.
            3. Answer with "Yes" or "No" based on the calculation. 
            4. Cite the source values for reference.
            5. Make sure final answer is consistent with calculation
            
        - If the answer is "No," **recalculate** and verify. If the same result appears, mark as final.  
        - If values are missing, answer "No" with reference: "Value not found."  
        
        --- OUTPUT FORMAT ---
        Only print raw JSON:  
            {{
              "q_la1": {{"answer": "Yes", "reference": "SubSum = 55, Most = 50"}},
              "q_la2": {{"answer": "No", "reference": "Group = 30, Exam = 40"}}
            }}  
        
        --- CAUTION ---  
        1. Do not assume values.  
        2. Do not carry over memory from previous runs.  
        3. Always recalculate each step explicitly.  

    '''
    
    # Send the message to the LLM model to process the question and return the response
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": f' Reference text: {extracted_text} Questions need answers: {questions} '}
    ]

    # Get answers from GPT
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or use gpt-4-1106-preview for better results
        messages=messages,
        temperature=0,
    )

    time.sleep(1.5)  # Adjust this value if needed
    answer = response.choices[0].message.content
    return answer


In [203]:
answers_la1_json=ask_questions(questions_la_1, extracted_text, answers_lapre)
print(answers_la1_json)

{
  "q_la1": {"answer": "No", "reference": "Value not found."},
  "q_la2": {"answer": "No", "reference": "Value not found."},
  "q_la3": {"answer": "No", "reference": "Value not found."},
  "q_la4": {"answer": "No", "reference": "Value not found."},
  "q_la5": {"answer": "No", "reference": "Value not found."}
}


In [204]:
#Flip the log in q_la4

answers_la1_json = ask_questions(questions_la_1, extracted_text, answers_lapre)

# Convert string to dict if needed
if isinstance(answers_la1_json, str):
    answers_la1_json = json.loads(answers_la1_json)

# Flip q_la4 logic
if "q_la4" in answers_la1_json:
    original_answer = answers_la1_json["q_la4"]["answer"]
    if original_answer == "Yes":
        answers_la1_json["q_la4"]["answer"] = "No"
    elif original_answer == "No":
        answers_la1_json["q_la4"]["answer"] = "Yes"

# Print or return final result
print(json.dumps(answers_la1_json, indent=2))

{
  "q_la1": {
    "answer": "No",
    "reference": "Value not found."
  },
  "q_la2": {
    "answer": "No",
    "reference": "Value not found."
  },
  "q_la3": {
    "answer": "No",
    "reference": "Value not found."
  },
  "q_la4": {
    "answer": "Yes",
    "reference": "Value not found."
  },
  "q_la5": {
    "answer": "No",
    "reference": "Value not found."
  }
}


In [205]:
answers_la1_dic=answers_la1_json
answers_la1_YN={key: value["answer"] for key, value in answers_la1_dic.items()}
answers_la1_YN=json.dumps(answers_la1_YN, indent=2)
print(answers_la1_YN)

{
  "q_la1": "No",
  "q_la2": "No",
  "q_la3": "No",
  "q_la4": "Yes",
  "q_la5": "No"
}


In [206]:
answers_la1_boolean=[]
answer_la1=json.loads(answers_la1_YN)
for value in answer_la1.values():
    if value.lower() == "yes":
        answers_la1_boolean.append(1)
    elif value.lower() == "no":
        answers_la1_boolean.append(0)
print(answers_la1_boolean)

[0, 0, 0, 1, 0]


In [207]:
# answers_la1_boolean = [1, 0, 1, 1, 1]

In [208]:
questions_la_2={
    "q_la6": "Are any of the exams in the course cumulative or comprehensive? - If there are no exams at all, respond 'No'.",
    "q_la7": "Does the syllabus include any method to assess students’ prior knowledge of course content, such as a pre-test, diagnostic quiz, or knowledge check activity?",
    "q_la8": "Does the course provide flexbility in grading, such as multiple attempts, flexibility in deadlines, dropping an assessment?", 
    "q_la9": "Are there any grades assigned to out-of-class discussion opportunities (e.g. discussion board, Packback, Piazza, etc)?",
    "q_la10": "Are there any grades assigned to in-class participation opportunitie (e.g. TopHat, Clicker, in-class quizzes, discussions, activities, short esseys)?", 
    "q_la11": "Is there any opportunity for self-check or practice quizzes and/or exams?",
    "q_la12": "Is there frequent knowledge checks (e.g. homeworks, quizzes, reflections, etc.) to assess learning beyond midterm and final exam and final projects?",
    "q_la13": "Are there any assingments that promote in-depth learning through reflection or writing?",
    "q_la14": "Is there any opportunity to earn extra credit in this class?",
    "q_la15": "Do students have choices in the format of assessment for some assingments in the course?", 
    "q_la16": "Are there any in-class peer discussion/learning opportunities and collaborative learning opportunities (e.g. group projects) indicated in the syllabus?"
}

In [209]:
def ask_questions(questions, text):


    for question in questions:
        prompt=f'''
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        Do not assume any context from previous documents. This is a fresh evaluation. All evidence must be contained in the provided text.
        '''
        messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Document Content: {extracted_text}\n Question about the document: {questions}"},
            ]
        #get answers from gpt
        response=client.chat.completions.create(
            model="gpt-4o-mini", # or use gpt-4-1106-preview for better result
            messages=messages,
            # seed=123,
            temperature=0,
        )
        time.sleep(1.5)  # Adjust this value if needed
        answer=response.choices[0].message.content

    return answer    
        

In [210]:
answers_la2_json=ask_questions(questions_la_2, extracted_text)
print(answers_la2_json)

{
    "q_la6": {"answer": "No", "reference": "Not found in the document."},
    "q_la7": {"answer": "No", "reference": "Not found in the document."},
    "q_la8": {"answer": "No", "reference": "Not found in the document."},
    "q_la9": {"answer": "No", "reference": "Not found in the document."},
    "q_la10": {"answer": "Yes", "reference": "In-class participation grades will be determined holistically over the semester."},
    "q_la11": {"answer": "No", "reference": "Not found in the document."},
    "q_la12": {"answer": "Yes", "reference": "Weekly assignments will provide practice on what has been discussed in the course."},
    "q_la13": {"answer": "Yes", "reference": "The course project will involve researching different physics curricula and developing a comprehensive proposal."},
    "q_la14": {"answer": "No", "reference": "Not found in the document."},
    "q_la15": {"answer": "No", "reference": "Not found in the document."},
    "q_la16": {"answer": "Yes", "reference": "The cou

In [211]:
answers_la2_dic=json.loads(answers_la2_json)
answers_la2_YN={key: value["answer"] for key, value in answers_la2_dic.items()}
answers_la2_YN=json.dumps(answers_la2_YN, indent=2)
print(answers_la2_YN)

{
  "q_la6": "No",
  "q_la7": "No",
  "q_la8": "No",
  "q_la9": "No",
  "q_la10": "Yes",
  "q_la11": "No",
  "q_la12": "Yes",
  "q_la13": "Yes",
  "q_la14": "No",
  "q_la15": "No",
  "q_la16": "Yes"
}


In [212]:
answers_la2_boolean=[]
answer_la2=json.loads(answers_la2_YN)
for value in answer_la2.values():
    if value.lower() == "yes":
        answers_la2_boolean.append(1)
    elif value.lower() == "no":
        answers_la2_boolean.append(0)
print(answers_la2_boolean)

[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1]


In [213]:
answers_la_boolean = answers_la1_boolean + answers_la2_boolean

In [214]:
questions_rt={
    "q_rt1": "Are course objectives clearly stated in the syllabus?",
    "q_rt2": "Are the course objectives supported by listed course topics?",
    "q_rt3": "Does this syllabus have a course schedule listing topics/readings/assignments in chronological or sequential order with due dates?",
    "q_rt4": "Does the syllabus have a course schedule easy to read and understand?", 
    "q_rt5": "Does the syllabus mention a grading rubric specifically for assignments like projects, papers, or discussions—not the overall course grade breakdown? A rubric counts even if it's referenced as available in Canvas.",
    "q_rt6": "Does it provide the breakdown of assessment categories and the percentage them composed of the final grade? The breakdown of grading category is not the same as grading rubric, it is not a rubric in this context.",
    "q_rt7": "Does instructor provide tips/advice for success in the class?",
    "q_rt8": "Is the format of the assessment explained? For quizzes or exams stated, such as multiple choices, fill in the blanks, short esseys, etc.? It also includes the other formats of essessment beyond quizzes or exams",
    "q_rt9": "Is there an option to only count the the better scored assessments, such as only selecting a number of highest scored quizzes or exams to count towards the final grades?",
    "q_rt10": "Is there clear policy about minimal absenses?",
    "q_rt11": "Does the course offer any flexibility for attendance which would allow students to make up lessons that was missed and have access to course slides?",
    "q_rt12": "Does the course allow students to make-up work for missed attendance?"
}

In [215]:
def ask_questions(questions, text):


    for question in questions:
        prompt=f'''
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        --- RESET NOTICE ---
        This is a fresh evaluation. You must not reuse previous answers. Carefully re-read the document and re-evaluate every question as if for the first time. Do not assume the previous answer is correct.

        '''

        messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Document Content: {extracted_text}\n Question about the document: {questions}"},
            ]
        #get answers from gpt
        response=client.chat.completions.create(
            model="gpt-4o-mini", # or use gpt-4-1106-preview for better result
            messages=messages,
            seed=123,
            temperature=0,
        )

        time.sleep(1.5)  # Adjust this value if needed
        answer=response.choices[0].message.content

    return answer    
        

In [216]:
answers_rt_json=ask_questions(questions_rt, extracted_text)
print(answers_rt_json)

{
    "q_rt1": {"answer": "No", "reference": "Not found in the document."},
    "q_rt2": {"answer": "No", "reference": "Not found in the document."},
    "q_rt3": {"answer": "No", "reference": "Not found in the document."},
    "q_rt4": {"answer": "No", "reference": "Not found in the document."},
    "q_rt5": {"answer": "Yes", "reference": "Grading in the class will be based on (1) in-class participation, (2) out-of-class assignments and (3) a course project."},
    "q_rt6": {"answer": "No", "reference": "Not found in the document."},
    "q_rt7": {"answer": "Yes", "reference": "If you think you might have trouble meeting a deadline, please reach out as early as possible."},
    "q_rt8": {"answer": "No", "reference": "Not found in the document."},
    "q_rt9": {"answer": "No", "reference": "Not found in the document."},
    "q_rt10": {"answer": "No", "reference": "Not found in the document."},
    "q_rt11": {"answer": "Yes", "reference": "If you need to miss a class for some reason, pl

In [217]:
answers_rt_dic=json.loads(answers_rt_json)
answers_rt_YN={key: value["answer"] for key, value in answers_rt_dic.items()}
answers_rt_YN=json.dumps(answers_rt_YN, indent=2)
print(answers_rt_YN)

{
  "q_rt1": "No",
  "q_rt2": "No",
  "q_rt3": "No",
  "q_rt4": "No",
  "q_rt5": "Yes",
  "q_rt6": "No",
  "q_rt7": "Yes",
  "q_rt8": "No",
  "q_rt9": "No",
  "q_rt10": "No",
  "q_rt11": "Yes",
  "q_rt12": "Yes"
}


In [218]:
answers_rt_boolean=[]
answer_rt=json.loads(answers_rt_YN)
for value in answer_rt.values():
    if value.lower() == "yes":
        answers_rt_boolean.append(1)
    elif value.lower() == "no":
        answers_rt_boolean.append(0)
print(answers_rt_boolean)

[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]


In [219]:
questions_lr={ 
    "q_lr1": "Does it contain the instructor's office hour or student hour?",
    "q_lr2": "Does it provide a rationale of why office hour or student hour exist?",
    "q_lr3": "Does the syllabus specify the instructor’s availability to meet with students outside of regular office hours, such as by appointment?",
    "q_lr4": "Does it have TA or GA or Teaching Assistant and their contact information listed?",
    "q_lr5": "Does it explain what the TA or GA or Teaching Asstant or Learning Assistant's role is?",
    "q_lr6": "Do TA or GA provide help for students in class?",
    "q_lr7": "Are academic resources listed, such as library services, tutoring, writing center, peer advisors, etc?"
}

In [220]:
def ask_questions(questions, text):


    for question in questions:
        prompt=f'''
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        Do not assume any context from previous documents. This is a fresh evaluation. All evidence must be contained in the provided text.
        '''


        messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Document Content: {extracted_text}\n Question about the document: {questions}"},
            ]
        #get answers from gpt
        response=client.chat.completions.create(
            model="gpt-4o-mini", # or use gpt-4-1106-preview for better result
            messages=messages,
            seed=123,
            temperature=0,
        )

        time.sleep(1.5)  # Adjust this value if needed
        answer=response.choices[0].message.content

    return answer    
        

In [221]:
answers_lr_json=ask_questions(questions_lr, extracted_text)
print(answers_lr_json)

{
    "q_lr1": {"answer": "Yes", "reference": "Office Hours: By Appointment (Book time with Hudson, Eric)"},
    "q_lr2": {"answer": "No", "reference": "Not found in the document."},
    "q_lr3": {"answer": "Yes", "reference": "Please don’t hesitate to reach out if you’d like to discuss anything. I’m happy to meet in person or on zoom."},
    "q_lr4": {"answer": "No", "reference": "Not found in the document."},
    "q_lr5": {"answer": "No", "reference": "Not found in the document."},
    "q_lr6": {"answer": "No", "reference": "Not found in the document."},
    "q_lr7": {"answer": "No", "reference": "Not found in the document."}
}


In [222]:
answers_lr_dic=json.loads(answers_lr_json)
answers_lr_YN={key: value["answer"] for key, value in answers_lr_dic.items()}
answers_lr_YN=json.dumps(answers_lr_YN, indent=2)
print(answers_lr_YN)

{
  "q_lr1": "Yes",
  "q_lr2": "No",
  "q_lr3": "Yes",
  "q_lr4": "No",
  "q_lr5": "No",
  "q_lr6": "No",
  "q_lr7": "No"
}


In [223]:
answers_lr_boolean=[]
answer_lr=json.loads(answers_lr_YN)
for value in answer_lr.values():
    if value.lower() == "yes":
        answers_lr_boolean.append(1)
    elif value.lower() == "no":
        answers_lr_boolean.append(0)
print(answers_lr_boolean)

[1, 0, 1, 0, 0, 0, 0]


In [224]:
#split this section into two ten question scale 
questions_cb1={
    "q_cb1": "Does the syllabus contain the instructor's pronouns?",
    "q_cb2": "Does the syllabus have TA or GA's pronounce listed?",
    "q_cb3": "Does the syllabus have an explaination of why pronous is important?",
    "q_cb4": "Does the syllabus have any resources listed to explain why pronous are important?",
    "q_cb5": "Are student life resources available to students, such as counseling services, bias incident report, diversity and inclusion office, LGBTQ resources, international student services?",
    "q_cb6": "Are cultural and religious holidays acknowledged or accommodated in the course (referring to holidays beyond University Academic Calendar, e.g. Yom Kippur, Ramadan, Lunar New Year, etc.)?",
    "q_cb7": "Is there an attempt to explain why course materials were chosen and/or how diverse authors/research content are included in course content?",
    "q_cb8": "Does the syllabus have a Diversity or Educational Equity statement?",
    "q_cb9": "Does the syllabus have a Land Acknowledgement statement?"
}

In [225]:
def ask_questions(questions, text):


    for question in questions:
        prompt=f'''
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        --- SEARCH STRATEGY ---
        When searching for:
        - **Diversity or Educational Equity statement** → Also search for similar statements like these:
          - "Diversity Commitment"
          - "Equity and Inclusion Statement"
          - "Social Justice Statement"
          - "Educational Equity Commitment"
        - **Land Acknowledgement statement** → Also search for:
          - "Territorial Acknowledgement"
          - "Indigenous Land Recognition"
          - "Acknowledgement of Indigenous Territory"
          - "We acknowledge the ancestral lands of..."
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        Do not assume any context from previous documents. This is a fresh evaluation. All evidence must be contained in the provided text.
        
        '''

        messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Document Content: {extracted_text}\n Question about the document: {questions}"},
            ]
        #get answers from gpt
        response=client.chat.completions.create(
            model="gpt-4o-mini", # or use gpt-4-1106-preview for better result
            messages=messages,
            seed=123,
            temperature=0,
        )
        
        time.sleep(1.5)  # Adjust this value if needed
        answer=response.choices[0].message.content

    return answer    
        

In [226]:
answers_cb1_json=ask_questions(questions_cb1, extracted_text)
print(answers_cb1_json)

{
    "q_cb1": {"answer": "No", "reference": "Not found in the document."},
    "q_cb2": {"answer": "No", "reference": "Not found in the document."},
    "q_cb3": {"answer": "No", "reference": "Not found in the document."},
    "q_cb4": {"answer": "No", "reference": "Not found in the document."},
    "q_cb5": {"answer": "Yes", "reference": "The document mentions counseling services and provides contact information for Counseling and Psychological Services."},
    "q_cb6": {"answer": "No", "reference": "Not found in the document."},
    "q_cb7": {"answer": "No", "reference": "Not found in the document."},
    "q_cb8": {"answer": "Yes", "reference": "The document includes a statement about fostering a diverse and inclusive environment and mentions acts of intolerance and discrimination."},
    "q_cb9": {"answer": "No", "reference": "Not found in the document."}
}


In [227]:
answers_cb1_dic=json.loads(answers_cb1_json)
answers_cb1_YN={key: value["answer"] for key, value in answers_cb1_dic.items()}
answers_cb1_YN=json.dumps(answers_cb1_YN, indent=2)
print(answers_cb1_YN)

{
  "q_cb1": "No",
  "q_cb2": "No",
  "q_cb3": "No",
  "q_cb4": "No",
  "q_cb5": "Yes",
  "q_cb6": "No",
  "q_cb7": "No",
  "q_cb8": "Yes",
  "q_cb9": "No"
}


In [228]:
answers_cb1_boolean=[]
answer_cb1=json.loads(answers_cb1_YN)
for value in answer_cb1.values():
    if value.lower() == "yes":
        answers_cb1_boolean.append(1)
    elif value.lower() == "no":
        answers_cb1_boolean.append(0)
print(answers_cb1_boolean)

[0, 0, 0, 0, 1, 0, 0, 1, 0]


In [229]:
#split this section into two ten question scale 
questions_cb2={
    "q_cb10": "Does the syllabus have the Disability Accommodations Statement?",
    "q_cb11": "Does the syllabus have the Acknowledgement of Intersectionality statement?",
    "q_cb12": "Does the syllabus have the Mandated Reporting or Bias Reporting statement?",
    "q_cb13": "Does the syllabus have the Non-discrimination statement?",
    "q_cb14": "Does the course offer any additional free course materials, or recommended reading list, or other online resources or resources in the library, etc?",
    "q_cb15": "Are individual sections separated by headers?",
    "q_cb16": "Overall, is the syllabus easy to follow? Meaning if there is a clear structure, titles of sections, language is easy to understand, logic is clear and content is not confusing",
    "q_cb17": "Is there a method used to assess students’ personal background (e.g., questionnaire, quiz, etc.)?",
    "q_cb18": "Is there a method used to assess students’ preferences for topics to be covered throughout the course (e.g., questionnaire, quiz, etc.)?",
    "q_cb19": "Is there a way for students to provide feedback about the course, such as polling, survey, questionaire about student feedback of the course?"
}


In [230]:
def ask_questions(questions, text):


    for question in questions:
        prompt=f'''
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        Do not assume any context from previous documents. This is a fresh evaluation. All evidence must be contained in the provided text.
        '''

        messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Document Content: {extracted_text}\n Question about the document: {questions}"},
            ]
        #get answers from gpt
        response=client.chat.completions.create(
            model="gpt-4o-mini", # or use gpt-4omini
            messages=messages,
            seed=123,
            temperature=0,
        )
         
        time.sleep(1.5)  # Adjust this value if needed
        answer=response.choices[0].message.content

    return answer    
        

In [231]:
answers_cb2_json=ask_questions(questions_cb2, extracted_text)
print(answers_cb2_json)

{
    "q_cb10": {"answer": "Yes", "reference": "The syllabus includes a Disability Policy that encourages students to discuss any issues preventing their success."},
    "q_cb11": {"answer": "No", "reference": "Not found in the document."},
    "q_cb12": {"answer": "No", "reference": "Not found in the document."},
    "q_cb13": {"answer": "Yes", "reference": "The syllabus states that acts of intolerance, discrimination, or harassment are not tolerated."},
    "q_cb14": {"answer": "No", "reference": "Not found in the document."},
    "q_cb15": {"answer": "Yes", "reference": "Sections are clearly separated by headers such as 'Course Description', 'Grading', 'In-Class Participation', etc."},
    "q_cb16": {"answer": "Yes", "reference": "The syllabus has a clear structure with titles of sections, easy-to-understand language, and logical content."},
    "q_cb17": {"answer": "No", "reference": "Not found in the document."},
    "q_cb18": {"answer": "No", "reference": "Not found in the docume

In [232]:
answers_cb2_dic=json.loads(answers_cb2_json)
answers_cb2_YN={key: value["answer"] for key, value in answers_cb2_dic.items()}
answers_cb2_YN=json.dumps(answers_cb2_YN, indent=2)
print(answers_cb2_YN)

{
  "q_cb10": "Yes",
  "q_cb11": "No",
  "q_cb12": "No",
  "q_cb13": "Yes",
  "q_cb14": "No",
  "q_cb15": "Yes",
  "q_cb16": "Yes",
  "q_cb17": "No",
  "q_cb18": "No",
  "q_cb19": "Yes"
}


In [233]:
answers_cb2_boolean=[]
answer_cb2=json.loads(answers_cb2_YN)
for value in answer_cb2.values():
    if value.lower() == "yes":
        answers_cb2_boolean.append(1)
    elif value.lower() == "no":
        answers_cb2_boolean.append(0)
print(answers_cb2_boolean)

[1, 0, 0, 1, 0, 1, 1, 0, 0, 1]


In [234]:
answers_cb_boolean = answers_cb1_boolean + answers_cb2_boolean

In [235]:
questions_ds={
    "q_ds1": "Are reading materials reflect any types of diverse population?",
    "q_ds2": "Are reading materials reflect work of under represented scholars? ",
    "q_ds3": "Is there a statement about the importance of studying diverse research literature and perspectives?",
    "q_ds4": "Is there an acknowledgement about the importance of historical context when reading research?",
    "q_ds5": "Is there an acknowledgement that the literature is often non-inclusive?"
}

In [236]:
def ask_questions(questions, text):


    for question in questions:
        prompt=f'''
        Instructions:
        You are a curriculum evaluator. Your task is to evaluate the elements of inclusive teaching in a syllabus by answering predefined questions. 
        You will need to read the text from beginning to end. You will respond to these questions related to elements of inclusive teaching with [Yes, No] as the final answer.
        A careful decision must be made, and no other output is accepted. Input questions will be in a JSON container. 
        Output answers will also be in a mandatory JSON format using only curly brackets. Do not include any other symbols or words in the answers.
        
        --- IMPORTANT --- 
        You are only assessing the **current document provided**.  
        1️⃣ You **must not** refer to or remember any previous document.  
        2️⃣ Your answers should be **exclusively based** on the evidence found in **this document**.  
        3️⃣ If you do not find the evidence, respond with "No" and cite "Not found in the document."  

        --- FORMAT RULES --- 
        - Answer each question with "Yes" or "No" based solely on the content of the document.  
        - For each answer, provide a `reference` that points directly to the content in the document that supports your decision.  
        - If the answer is "No," still provide a `reference` indicating that the evidence is not present.  

        
        Example output format:
            Input: {{"question 1": "Is the language in this document inclusive?", "question 2": "Are there additional free materials listed for the course?",...}}
            Output: {{"q_eo1": {{"answer": "Yes", "reference": "Textbook reference or details here"}}, "q_eo2": {{"answer": "No", "reference": "Details if any or 'None'"}},...}}

        
        --- VERIFICATION STEP ---  
        After processing each question, **double-check the evidence found** in the document to ensure it matches your answer.  
        - If you identify any inconsistency or error during verification, **correct it immediately** before moving to the next question.  
        - Only provide final output when you are confident the answers are accurate and directly evidence-based.

        --- CAUTION ---  
        Do not assume any context from previous documents. This is a fresh evaluation. All evidence must be contained in the provided text.
        '''

        messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Document Content: {extracted_text}\n Question about the document: {questions}"},
            ]
        #get answers from gpt
        response=client.chat.completions.create(
            model="gpt-4o-mini", # or use gpt-4-1106-preview for better result
            messages=messages,
            seed=123,
            temperature=0,
        )

        time.sleep(1.5)  # Adjust this value if needed
        answer=response.choices[0].message.content

    return answer    
        

In [237]:
answers_ds_json=ask_questions(questions_ds, extracted_text)
print(answers_ds_json)

{
    "q_ds1": {"answer": "No", "reference": "Not found in the document."},
    "q_ds2": {"answer": "No", "reference": "Not found in the document."},
    "q_ds3": {"answer": "No", "reference": "Not found in the document."},
    "q_ds4": {"answer": "No", "reference": "Not found in the document."},
    "q_ds5": {"answer": "No", "reference": "Not found in the document."}
}


In [238]:
answers_ds_dic=json.loads(answers_ds_json)
answers_ds_YN={key: value["answer"] for key, value in answers_ds_dic.items()}
answers_ds_YN=json.dumps(answers_ds_YN, indent=2)
print(answers_ds_YN)

{
  "q_ds1": "No",
  "q_ds2": "No",
  "q_ds3": "No",
  "q_ds4": "No",
  "q_ds5": "No"
}


In [239]:
answers_ds_boolean=[]
answer_ds=json.loads(answers_ds_YN)
for value in answer_ds.values():
    if value.lower() == "yes":
        answers_ds_boolean.append(1)
    elif value.lower() == "no":
        answers_ds_boolean.append(0)
print(answers_ds_boolean)

[0, 0, 0, 0, 0]


In [240]:
#final boolean by category for recalculation of percentages

# answers_eo_boolean=[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
# answers_ex_boolean=[1, 1, 1, 1, 1, 0]
# answers_gm_boolean=[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1]
# answers_la_boolean=[1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
# answers_rt_boolean=[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0]
# answers_lr_boolean=[1, 1, 1, 1, 1, 1, 1]
# answers_cb_boolean=[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
# answers_ds_boolean=[0, 0, 0, 0, 0]


In [241]:
boolean_answers = answers_eo_boolean + answers_ex_boolean + answers_gm_boolean + answers_la_boolean + answers_rt_boolean + answers_lr_boolean + answers_cb_boolean + answers_ds_boolean

In [242]:
print(boolean_answers)

[0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]


In [243]:
with open(r'C:\Users\lixia\OneDrive - The Pennsylvania State University\School\Dissertation\Instructor Lit\Phase I Interview\ChatGPT\Outfile.csv', "w") as outfile:
    for i in boolean_answers:
        outfile.write(f"{i}\n")

In [244]:
answer_df.insert(loc=4, column='Values', value=boolean_answers)

In [257]:
# boolean_answers=df_psych231['Values'].to_list()

In [258]:
#psych231 boolean by category 

# answers_eo_boolean=boolean_answers[0: 11]
# answers_ex_boolean=boolean_answers[12: 17]
# answers_gm_boolean=boolean_answers[18: 27]
# answers_la_boolean=boolean_answers[28: 43]
# answers_rt_boolean=boolean_answers[44: 55]
# answers_lr_boolean=boolean_answers[56: 62]
# answers_cb_boolean=boolean_answers[63: 67]
# answers_in_boolean=boolean_answers[68: 81]
# answers_ds_boolean=boolean_answers[82: 86]

In [245]:
answer_df.to_csv(r'C:\Users\lixia\OneDrive - The Pennsylvania State University\School\Dissertation\Instructor Lit\Phase I Interview\ChatGPT\Out.csv', index=False)

In [ ]:
#define dictionary 
categories={
    "eo": answers_eo_boolean,
    "ex": answers_ex_boolean,
    "gm": answers_gm_boolean,
    "la": answers_la_boolean,
    "rt": answers_rt_boolean, #change it to rt later
    "lr": answers_lr_boolean,
    "cb": answers_cb_boolean,
    "ds": answers_ds_boolean
}

In [ ]:
def calculate_percengate(values):
    total=len(values)
    count_ones=values.count(1)
    percentage_ones=count_ones/total
    percentage_zeros=1-percentage_ones
    return round(percentage_ones, 2), round(percentage_zeros, 2)

In [ ]:
percentage_ones_dict={}
percentage_zeros_dict={} 

In [ ]:
for category, values in categories.items(): #categories.items refers to key and value
    percentage_ones, percentage_zeros = calculate_percengate(values)
    percentage_ones_dict[category]=percentage_ones
    percentage_zeros_dict[category]=percentage_zeros

In [ ]:
print(percentage_ones_dict)
print(percentage_zeros_dict)

In [ ]:
percentage_ones=list(percentage_ones_dict.values())
percentage_zeros=list(percentage_zeros_dict.values())

In [ ]:
print(percentage_ones)
print(percentage_zeros)

In [ ]:


# write percengates to of numbers

# Specify the file name
file_name = r'C:\Users\lixia\OneDrive - The Pennsylvania State University\School\Dissertation\Instructor Lit\Phase I Interview\ChatGPT\percent.json'
percent_dict={
    "percentage_ones": percentage_ones,
    "percentage_zeros": percentage_zeros
}
# Write the list to a file in JSON format
with open(file_name, 'w') as file:
    json.dump(percent_dict, file)

print(f"Numbers have been written to {file_name}")

In [ ]:
yes_percentages_v2=[proportion * 100 for proportion in percentage_ones]
no_percentages_v2=[proportion * 100 for proportion in percentage_zeros]

In [ ]:
#use regular expression to extract file name
match=re.search(r'\\([^\\]+)\.pdf$', pdf_file_path)

if match: 
    course_title = match.group(1)
    print(course_title)
else: 
    print("Course Title Not Found")

In [ ]:
print(match)

In [ ]:
categories = ['Engagement Opportunities', 'Exploration', 'Growth Mindset', 'Learning Assessment', 'Requirements Transparency', 'Learning Resources', 'Cultivate Belonging', 'Inclusivity', 'Diversity Scholarships']

In [ ]:
df_course=pd.DataFrame(
    {
        "category": categories,
        "yes_percentage": yes_percentages_v2,
        "no_percentage": no_percentages_v2,
        "course": course_title
    }
)

In [ ]:
df_course=df_course.replace(to_replace={'course':'PSYCH231'}, value='PSYCH231')
df_course

In [ ]:
df_course.to_csv(r'C:\Users\lixia\OneDrive - The Pennsylvania State University\School\Dissertation\Instructor Lit\Phase I Interview\ChatGPT\course.csv', index=False)

In [1]:
import csv
import json
import firebase_admin
from firebase_admin import credentials, db

In [2]:
# Path to the service account key file
cred = credentials.Certificate(r"C:\Users\lixia\Firebase\inclusight-6e55a-firebase-adminsdk-vv1dg-d4cf053cda.json")

# Initialize the Firebase app (for Firestore and Realtime Database)
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://inclusight-6e55a-default-rtdb.firebaseio.com/'  # Realtime Database URL
})


In [1]:
jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --inplace your_notebook.ipynb


SyntaxError: invalid syntax (586152860.py, line 1)